In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
obj_name = "bunny"

base_dir = Path("datasets")
data_dir = base_dir / obj_name
sfm_dir = data_dir / "sfm"
cam_dir = sfm_dir / "cam"
recon_dir = data_dir / "recon"

pcd_path = recon_dir / "reconstructed_filtered.pcd"
cam_path = cam_dir / "Camera.002.npz"

In [3]:
# load camera
with np.load(cam_path) as X:
    K, RT = [X[i].astype(np.float64) for i in ("K", "RT")]
print(K)
RT2 = np.eye(4)
RT2[:3,:] = RT
print(RT2)

[[877.91237385   0.         300.        ]
 [  0.         877.91237385 400.        ]
 [  0.           0.           1.        ]]
[[ 0.87400233  0.2152691  -0.43563647  2.59538966]
 [-0.23046462  0.97290697  0.01838733  0.58875611]
 [ 0.42779199  0.08432822  0.89993487 -0.49766862]
 [ 0.          0.          0.          1.        ]]


In [4]:
pcd = o3d.io.read_point_cloud(str(pcd_path))
print(pcd)

PointCloud with 6035 points.


In [4]:
bunny = o3d.data.BunnyMesh()
gt_mesh = o3d.io.read_triangle_mesh(bunny.path)
gt_mesh.compute_vertex_normals()

pcd = gt_mesh.sample_points_poisson_disk(5000)
o3d.visualization.draw_geometries([pcd])

In [5]:
# invalidate existing normals
pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))
pcd.estimate_normals()
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

In [6]:
#pcd.orient_normals_towards_camera_location(camera_location=RT[:,3])
pcd.orient_normals_consistent_tangent_plane(100)
o3d.visualization.draw_geometries([pcd], point_show_normal=True)
print(type(pcd.normals))

<class 'open3d.cuda.pybind.utility.Vector3dVector'>


In [56]:
radii = [0.005, 0.01, 0.02, 0.04]
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([pcd, mesh])

In [7]:
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
#print(mesh)
o3d.visualization.draw_geometries([mesh])

[Open3D DEBUG] Input Points / Samples: 5000 / 5000
[Open3D DEBUG] #   Got kernel density: 0.0214829 (s), 325.148 (MB) / 325.148 (MB) / 325 (MB)
[Open3D DEBUG] #     Got normal field: 0.0324948 (s), 329.512 (MB) / 329.512 (MB) / 329 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 5.965973e-04 / 2.982986e+00
[Open3D DEBUG] #       Finalized tree: 0.029947 (s), 334.984 (MB) / 334.984 (MB) / 334 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.037766 (s), 334.984 (MB) / 334.984 (MB) / 334 (MB)
[Open3D DEBUG] #Set point constraints: 0.00998998 (s), 334.984 (MB) / 334.984 (MB) / 334 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 288534 / 107328 / 222425
[Open3D DEBUG] Memory Usage: 334.984 MB
Cycle[0] Depth[0/9]:	Updated constraints / Got system / Solved in:  0.000 /  0.000 /  0.000	(334.984 MB)	Nodes: 8
CG: 1.0735e+00 -> 1.0735e+00 -> 2.4452e-06 (2.3e-06) [0]
Cycle[0] Depth[1/9]:	Updated constraints / Got system / Solved in:  0.001 /  0.000 /  0.000	(334.984 MB)	Nodes: 27
  GS

In [39]:
print("Try to render a mesh with normals (exist: " +
      str(mesh.has_vertex_normals()) + ") and colors (exist: " +
      str(mesh.has_vertex_colors()) + ")")

Try to render a mesh with normals (exist: True) and colors (exist: True)


In [8]:
mesh.compute_vertex_normals()
print(np.asarray(mesh.triangle_normals))
o3d.visualization.draw_geometries([mesh])

[[-0.73535296 -0.14361035 -0.66229306]
 [-0.73472056 -0.14935774 -0.66172348]
 [-0.70393026 -0.13016927 -0.69823933]
 ...
 [ 0.49503108 -0.23205222  0.83731475]
 [ 0.52253832 -0.30059474  0.79786998]
 [ 0.38438426 -0.22553974  0.89519862]]


In [16]:
print('visualize densities')
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
o3d.visualization.draw_geometries([density_mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

visualize densities


In [9]:
cx = K[0,2]
cy = K[1,2]
img_width = int(cx * 2)
img_height = int(cy * 2)
cx -= 0.5
cy -= 0.5
print(img_width, img_height)
fx = K[0,0]
fy = K[1,1]

vis = o3d.visualization.Visualizer()
vis.create_window(width=img_width, height=img_height)
vis.get_render_option().mesh_color_option = o3d.visualization.MeshColorOption.Normal
vis.add_geometry(mesh)

view_ctl = vis.get_view_control()
#assert id(view_ctl) == id(vis.get_view_control())


cam = view_ctl.convert_to_pinhole_camera_parameters()
#RT2[2,3] = 10.00
cam.extrinsic = RT2
#pinhole = o3d.camera.PinholeCameraIntrinsic(img_width, img_height, K)
cam.intrinsic.set_intrinsics(img_width, img_height, fx, fy, cx, cy)
res = view_ctl.convert_from_pinhole_camera_parameters(cam)
print(res)


current_param = view_ctl.convert_to_pinhole_camera_parameters()
print(current_param.intrinsic.intrinsic_matrix)
print(current_param.extrinsic)

vis.run()
vis.destroy_window()

600 800
True
[[692.82032303   0.         299.5       ]
 [  0.         692.82032303 399.5       ]
 [  0.           0.           1.        ]]
[[ 0.87400233  0.2152691  -0.43563647  2.59538966]
 [-0.23046462  0.97290697  0.01838733  0.58875611]
 [ 0.42779199  0.08432822  0.89993487 -0.49766862]
 [ 0.          0.          0.          1.        ]]


In [60]:
from open3d.visualization import rendering

In [ ]:
cx = K[0,2]*2
cy = K[1,2]*2
img_width = cx * 2
img_height = cy * 2
fx = K[0,0]
fy = K[1,1]

# Create a renderer with a set image width and height
render = rendering.OffscreenRenderer(img_width, img_height)

# setup camera intrinsic values
pinhole = o3d.camera.PinholeCameraIntrinsic(img_width, img_height, fx, fy, cx, cy)
    
# Pick a background colour of the rendered image, I set it as black (default is light gray)
render.scene.set_background([1.0, 1.0, 1.0, 1.0])  # RGBA

# # now create your mesh
# mesh = o3d.geometry.TriangleMesh()
# mesh.paint_uniform_color([1.0, 0.0, 0.0]) # set Red color for mesh 
# # define further mesh properties, shape, vertices etc  (omitted here)  

# # Define a simple unlit Material.
# # (The base color does not replace the mesh's own colors.)
# mtl = o3d.visualization.rendering.Material()
# mtl.base_color = [1.0, 1.0, 1.0, 1.0]  # RGBA
# mtl.shader = "defaultUnlit"

# add mesh to the scene
render.scene.add_geometry("MyMeshModel", mesh, mtl)

# render the scene with respect to the camera
render.scene.camera.set_projection(pinhole, 0.1, 1.0, 640, 480)
img_o3d = render.render_to_image()

# we can now save the rendered image right at this point 
o3d.io.write_image("output.png", img_o3d, 9)